In [1]:
import openai
import requests
import json
from typing import List, Dict, Any

# =============================================================================
# SETUP AND CONFIGURATION
# =============================================================================

# Ollama typically runs on localhost:11434
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_API_URL = f"{OLLAMA_BASE_URL}/v1"

# Configure OpenAI client to use Ollama
client = openai.OpenAI(
    base_url=OLLAMA_API_URL,
    api_key="ollama"  # Ollama doesn't require a real API key, but OpenAI client expects one
)


In [2]:
# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def check_ollama_status():
    """Check if Ollama is running and accessible"""
    try:
        response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
        if response.status_code == 200:
            return True, "Ollama is running successfully!"
        else:
            return False, f"Ollama responded with status code: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return False, f"Cannot connect to Ollama: {e}"

def list_available_models():
    """List all available models in Ollama"""
    try:
        response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
        if response.status_code == 200:
            models = response.json().get('models', [])
            return [model['name'] for model in models]
        else:
            return []
    except requests.exceptions.RequestException:
        return []

def pull_model(model_name: str):
    """Pull a model from Ollama registry"""
    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/pull",
            json={"name": model_name}
        )
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False


In [3]:
print("🔍 Checking Ollama connection...")
is_running, status_message = check_ollama_status()
print(f"Status: {status_message}")


🔍 Checking Ollama connection...
Status: Ollama is running successfully!


In [5]:
if is_running:
    available_models = list_available_models()
    print(f"\n📦 Available models: {available_models}")
    
    if not available_models:
        print("\n⚠️  No models found! You may need to pull a model first.")
        print("Example: Run 'ollama pull llama2' in your terminal")
else:
    print("\n❌ Please start Ollama first:")
    print("1. Install Ollama from https://ollama.ai")
    print("2. Run 'ollama serve' in terminal")
    print("3. Pull a model: 'ollama pull llama2' or 'ollama pull mistral'")


📦 Available models: ['llama3.2:latest']


In [6]:
def simple_chat_example(model_name: str = "llama2"):
    """Basic chat completion example"""
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Explain quantum computing in simple terms."}
            ],
            max_tokens=200,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

In [7]:
if available_models:
    model_to_use = available_models[0]  # Use first available model
    print(f"\n🤖 Using model: {model_to_use}")
    result = simple_chat_example(model_to_use)
    print(f"\nResponse:\n{result}")


🤖 Using model: llama3.2:latest

Response:
Quantum computing is a new way of processing information that's different from classical computers like your smartphone or laptop.

**Classical Computers**

Classical computers use "bits" to store and process information. Bits can be either 0 or 1, and they're used to perform calculations by using simple arithmetic operations (like addition or subtraction).

**Quantum Computers**

Quantum computers, on the other hand, use "qubits" (quantum bits). Qubits are special because they can be both 0 AND 1 at the same time! This is known as a superposition.

Imagine you have a coin that can either be heads or tails. A classical computer would say it's either one or the other, but a quantum computer would say it's both heads AND tails at the same time (until you measure it).

**Entanglement**

Another key feature of quantum computers is entanglement. When two qubits are connected (entangled), what happens to one qubit instantly


In [8]:

def streaming_chat_example(model_name: str, user_message: str):
    """Example of streaming chat completion"""
    try:
        stream = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful coding assistant."},
                {"role": "user", "content": user_message}
            ],
            stream=True,
            max_tokens=300,
            temperature=0.7
        )
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Example usage:
if available_models:
    print("\n🔄 Streaming example:")
    streaming_chat_example(available_models[0], "Write a Python function to calculate factorial")



🔄 Streaming example:
**Factorial Function in Python**

Here's a simple recursive and iterative function to calculate the factorial of a given number.

### Recursive Implementation

```python
def factorial_recursive(n):
    """
    Calculates the factorial of a given number using recursion.
    
    Args:
        n (int): The input number.
    
    Returns:
        int: The factorial of n.
    
    Raises:
        ValueError: If n is negative.
    """
    if not isinstance(n, int):
        raise TypeError("Input must be an integer.")
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    elif n == 0 or n == 1:
        return 1
    else:
        return n * factorial_recursive(n-1)
```

### Iterative Implementation

```python
def factorial_iterative(n):
    """
    Calculates the factorial of a given number using iteration.
    
    Args:
        n (int): The input number.
    
    Returns:
        int: The factorial of n.
    
    Raises:
        Va

In [9]:
class OllamaConversation:
    """Handle multi-turn conversations with Ollama"""
    
    def __init__(self, model_name: str, system_message: str = "You are a helpful assistant."):
        self.model_name = model_name
        self.messages = [{"role": "system", "content": system_message}]
        self.client = client
    
    def add_user_message(self, content: str):
        """Add a user message to the conversation"""
        self.messages.append({"role": "user", "content": content})
    
    def get_response(self, stream: bool = False, **kwargs):
        """Get response from the model"""
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=self.messages,
                stream=stream,
                **kwargs
            )
            
            if stream:
                full_response = ""
                for chunk in response:
                    if chunk.choices[0].delta.content is not None:
                        content = chunk.choices[0].delta.content
                        print(content, end="", flush=True)
                        full_response += content
                self.messages.append({"role": "assistant", "content": full_response})
                return full_response
            else:
                response_content = response.choices[0].message.content
                self.messages.append({"role": "assistant", "content": response_content})
                return response_content
                
        except Exception as e:
            return f"Error: {e}"
    
    def reset_conversation(self, system_message: str = "You are a helpful assistant."):
        """Reset the conversation"""
        self.messages = [{"role": "system", "content": system_message}]
    
    def get_conversation_history(self):
        """Get the full conversation history"""
        return self.messages


In [10]:
if available_models:
    print(f"\n💬 Starting conversation with {available_models[0]}")
    
    # Initialize conversation
    conv = OllamaConversation(
        model_name=available_models[0],
        system_message="You are a Python programming tutor. Be concise and helpful."
    )
    
    # Example conversation
    conv.add_user_message("What is a list comprehension in Python?")
    response1 = conv.get_response(max_tokens=150)
    print(f"Assistant: {response1}\n")
    
    conv.add_user_message("Can you give me a practical example?")
    response2 = conv.get_response(max_tokens=150)
    print(f"Assistant: {response2}\n")
    
    # Print conversation history
    print("📝 Conversation History:")
    for i, msg in enumerate(conv.get_conversation_history()):
        print(f"{i+1}. {msg['role'].title()}: {msg['content'][:100]}...")


💬 Starting conversation with llama3.2:latest
Assistant: **List Comprehensions in Python**

A list comprehension is a compact way to create lists in Python. It consists of brackets containing an expression followed by a `for` clause, then zero or more `for` or `if` clauses. The result is a new list resulting from evaluating the expression in the context of the given control structures.

**Basic Syntax**
```python
[expression for variable in iterable if condition]
```
**Example**
```python
numbers = [1, 2, 3, 4, 5]
even_numbers = [num for num in numbers if num % 2 == 0]
print(even_numbers)  # Output: [2, 4]
```
In this example, the

Assistant: **Practical Example: Processing a List of Students**

Suppose we have a list of students with their names and ages. We want to extract the names of all students who are older than 18.

```python
# Original list of students
students = [
    {"name": "John", "age": 19},
    {"name": "Alice", "age": 17},
    {"name": "Bob", "age": 20},
    {"name": "